In [11]:
'''
sigmaX 높을수록 뿌옇게 변함(비식별화 잘 됨)
'''

# 모듈 로딩
import cv2
from yolov5facedetector.face_detector import YoloDetector
import time
import math
import re

url = r'C:\Users\USER\Desktop\vs코드\wkit_vs\mosaic\mosaic_1123\test_01.mp4'


def video_save(webcam, file_name):
    '''
    영상 저장을 위한 객체 생성
    '''
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = round(webcam.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = round(webcam.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = webcam.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(file_name, fourcc, fps, (width, height))  # 파일명 경로

    return fps, out


def mosaic(url, yolo_type='yolov5n',  target_size=480, gpu=0, min_face=0, conf_thres=0.3, iou_thres=0.5, sigma=55):
    
    model = YoloDetector(yolo_type=yolo_type, target_size=target_size, gpu=gpu, min_face=min_face) 
    webcam = cv2.VideoCapture(url)
    
    if '\\' in url:
        file_name = re.sub('.mp4|.avi','',url.split('\\')[-1])
    else:
        file_name = re.sub('.mp4|.avi','',url.split('/')[-1])
        
    fps, out = video_save(webcam, f"{file_name}_mosaic.mp4")

    file_names = f"{file_name}_mosaic.mp4"

    # 동영상 파일 열기 성공 여부 확인
    if not webcam.isOpened():
        print("Could not open webcam") 
        exit()
        
    # mosa=True일 때 영상의 기본 상태 : 모자이크
    mosa = True

    fps_value = 1./fps  # 원본 fps와 맞추기 위해 나눌 수

    gauge_bar = -1   # 게이지 바 초기화

    f = open('./test.txt', 'w')  # 파일 열기
    
    f.write('frame_num\tbboxes\tpeople_num\ttime\tframe\n')  # 파일 컬럼명

    f = open('./test.txt', 'a')  # 파일 열기

    while webcam.isOpened():
            
        status, frame = webcam.read()

        # 알고리즘 시작 지점
        start_time = time.time()

        if frame is None: break

        if (webcam.get(cv2.CAP_PROP_POS_FRAMES) == 1) | (webcam.get(cv2.CAP_PROP_POS_FRAMES) == 2) | (webcam.get(cv2.CAP_PROP_POS_FRAMES) % fps_value == 0):
            bboxes, confs, points = model.predict(frame, conf_thres=conf_thres, iou_thres=iou_thres)
            
            for idx in range(len(bboxes[0])):

                f.write('%-3d \t' %int(webcam.get(cv2.CAP_PROP_POS_FRAMES)))  # frame_num
                f.write('%-22s \t' %bboxes[0][idx])  # bboxes
                f.write('%-3d \t' %len(bboxes[0]))  # people_num
                f.write("%12s \t" %time.strftime('%Y-%m-%d %H:%M:%S'))  # time
                f.write('%s \n' %frame.flatten())  # frame   # 1차원으로 바꾼 거라서 사용하려면 형태 reshape 해야 함

        else: pass


        if not status:
            print("Could not read frame")
            exit()

        key = cv2.waitKey(1)
        if key == 26:  # Ctrl + Z : 모자이크 켜짐
            mosa = True
        elif key == 24:  # Ctrl + X : 모자이크 꺼짐
            mosa = False

        for bbox in bboxes[0]:
            (startX, startY)=bbox[0], bbox[1]
            (endX, endY)=bbox[2], bbox[3]

            if mosa == True:
                face_region = frame[startY:endY, startX:endX]  # 관심영역(얼굴) 지정
                
                # 모자이크
                '''
                cv2.GaussianBlur(src, ksize, sigmaX, dst=None, sigmaY=None, borderType=None) -> dst

                • src: 입력 영상. 각 채널 별로 처리됨.
                • dst: 출력 영상. src와 같은 크기, 같은 타입.
                • ksize: 가우시안 커널 크기. (0, 0)을 지정하면 sigma 값에 의해 자동 결정됨
                • sigmaX: x방향 sigma.
                • sigmaY: y방향 sigma. 0이면 sigmaX와 같게 설정.
                • borderType: 가장자리 픽셀 확장 방식.
                '''
                frame[startY:endY, startX:endX] = cv2.GaussianBlur(face_region, ksize=(0,0), sigmaX=sigma)

        
        # display output

        out.write(frame)   # 동영상 저장

        cv2.imshow("Mosaic Video", frame)  # 윈도우 창에 이미지를 띄움

        # 알고리즘 종료 시점
        # print('모자이크 처리에 걸리는 시간 \n▶ FPS', int(1./(time.time() - start_time)),\
        #     '\n▶ Time:',  time.time() - start_time, '\n')
        
        if webcam.get(cv2.CAP_PROP_POS_FRAMES) == 3:

            fps_value = math.ceil((time.time() - start_time)/(1./fps))

        
        if gauge_bar != round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT)):

            print(' '*round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT))+'▽',\
                round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT)),'%')
            print('[', end='')
            print(round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT))*'■'+\
                (100-round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT)))*' ', end='')
            print(']')
            gauge_bar = round(webcam.get(cv2.CAP_PROP_POS_FRAMES)*100/webcam.get(cv2.CAP_PROP_FRAME_COUNT))
            
        if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키 입력 받으면 윈도우 창이 종료
            break

    f.close()   # 파일 닫기

    #동영상 파일을 닫고 메모리 해제
    out.release()
    webcam.release()  


    # 모든 윈도우 창을 닫음
    cv2.destroyAllWindows()

    return file_names


file_names = mosaic(url)

▽ 0 %
[                                                                                                    ]
 ▽ 1 %
[■                                                                                                   ]
  ▽ 2 %
[■■                                                                                                  ]
   ▽ 3 %
[■■■                                                                                                 ]
    ▽ 4 %
[■■■■                                                                                                ]
     ▽ 5 %
[■■■■■                                                                                               ]
      ▽ 6 %
[■■■■■■                                                                                              ]
       ▽ 7 %
[■■■■■■■                                                                                             ]
        ▽ 8 %
[■■■■■■■■                                                                             

In [12]:
import os
from moviepy.editor import *
import ffmpeg

def sound():
    '''
    모자이크 영상에 음성 추출 함수
    '''
    try:
        # 원본 영상에서 음성 추출
        ffmpeg.input(file_names.split('_mosaic')[0]+'.mp4').output(file_names.split('_mosaic')[0]+'.mp3').run()

        # 모자이크 영상과 원본 음성 합치기
        videoclip = VideoFileClip(file_names)
        audioclip = AudioFileClip(file_names.split('_mosaic')[0]+'.mp3')

        videoclip.audio = audioclip
        videoclip.write_videofile('end_'+file_names)

    except:
        os.rename(file_names, 'end_'+file_names)

In [13]:
'''
crf 낮을수록 고화질, 용량 높아짐
'''

# 모듈 로딩
import ffmpeg
import os


def compact_save(file_names=file_names, PATH='./', crf=25):
    
    sound()
    
    # 모자이크 용량 줄이기
    try:
        os.remove(file_names)  # 음성 없는 모자이크 영상 삭제
        os.remove(file_names.split('_mosaic')[0]+'.mp3')  # 원본 음성 삭제
    
    except: pass

    file_names = 'end_'+file_names
    
    if os.path.isdir('resized') == False:
        
        # ffmpeg으로 변환할 때는 기존 파일에 덮어쓰기가 안되므로 변환된 파일들을 저장할 폴더를 생성
        os.mkdir('resized')

    # 변환 중 에러가 발생하는 경우가 있어 에러 처리를 위한 리스트를 만듦
    err = []

        # (try) 파일 변환을 시도함.
        # (except) 에러가 날 경우 변환 중이던 파일을 지우고 파일명을 err 리스트에 추가함

    try:
        # crf[quality] : 비트레이트 대신 화질 기준으로 인코딩할 때 쓰는 옵션. libx264 코덱 기준 사용 가능 범위 0-51, 0은 무손실, 디폴트는 23
        ffmpeg.input(PATH+file_names).output('resized/'+file_names, crf=crf, vsync='vfr').run()

    except:
        os.remove('resized/'+file_names)
        err.append(file_names)

    # 파일이 resized 폴더 내에 있다면(에러가 나지 않고 변환되어 옮겨져 있다면)
    # 원래 폴더에서 파일을 삭제함
    if file_names in os.listdir('resized'):
        os.remove(PATH+file_names)

    # mp4 파일 목록을 순화하면서 오류 났던 파일 (err 리스트)을
    # 제외한 나머지 파일을 원래 폴더로 모두 옮김

    if file_names in err:
        pass
    else:
        os.replace('resized/'+file_names, PATH+file_names)

    # resized 폴더를 삭제함
    os.rmdir('resized')
    

    # 변환에 실패한 파일 목록을 출력함
    # print('실패한 목록: \n', err)

compact_save()

Moviepy - Building video end_test_01_mosaic.mp4.
MoviePy - Writing audio in end_test_01_mosaicTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video end_test_01_mosaic.mp4



t:  50%|█████     | 18/36 [00:00<00:00, 177.39it/s, now=None]WARNING:py.warnings:c:\Users\USER\anaconda3\envs\ff38\lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file test_01_mosaic.mp4, 2764800 bytes wanted but 0 bytes read,at frame 35/36, at time 1.17/1.17 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready end_test_01_mosaic.mp4


In [4]:
import pandas as pd

df = pd.read_csv('test.txt', sep='\t')
df

,frame_num,bboxes,people_num,time,frame
0,1,"[518, 92, 691, 313]",1,2022-11-23 16:26:59,[0 0 4 ... 0 4 4]
1,2,"[517, 91, 689, 314]",1,2022-11-23 16:26:59,[0 0 4 ... 0 4 4]
2,4,"[515, 95, 685, 318]",1,2022-11-23 16:27:00,[0 0 4 ... 0 4 4]
3,8,"[512, 103, 680, 324]",1,2022-11-23 16:27:00,[0 0 4 ... 0 4 4]
4,12,"[506, 111, 672, 330]",1,2022-11-23 16:27:01,[0 0 4 ... 0 4 4]
...,...,...,...,...,...
56,184,"[399, 148, 705, 529]",1,2022-11-23 16:27:20,[ 0 0 0 ... 15 2 0]
57,188,"[399, 142, 700, 525]",1,2022-11-23 16:27:21,[ 0 0 0 ... 15 2 0]
58,192,"[400, 135, 703, 524]",1,2022-11-23 16:27:22,[ 0 0 0 ... 15 2 0]
59,196,"[406, 125, 716, 532]",1,2022-11-23 16:27:23,[ 0 0 0 ... 15 2 0]
